In [1]:
# imports
import math

# Canal cobra 1$ x persona.
# 10,000,000$ para invertir en publicidad.

# TP 3: MKT Publicidades

#### Integrantes
- Mirra, Ariel Marcelo.
- Ricci, Juan Gabriel.

#### Docentes
- Olmedo, Agustin.
- Gioia, Alicia.

### Resumen
TODO

### Introduccion
Este trabajo consiste en el uso de las herramientas aprendidas en clase (Runge-Kutta, matplotlib, etc.) para la resolucion de un problema real: medir el impacto de una publicidad utilizando la ecuación de la curva logística.

### 0) Importe de datos
Se obtendran los datos de los archivos 'coeficientes_canal.csv' (valores de k para cada canal) y 'televidentes_canal_horario.csv' (valores de M para cada canal y horario).

In [16]:
# Coeficiente K: 
coef_k = []
f = open("coeficientes_canal.csv", "r")
f.readline()  # first line is not desired, bypassing it.
for row in f:
    coef_k.append(row.split(","))
f.close()
            
# Televidentes
tv_viewers_channels = []
f = open("televidentes_canal_horario.csv", "r")
for row in f:
    tv_viewers_channels.append(row.split(","))
f.close()

### 1) Implementar la ecuación de la curva logística
Se hara uso de los 'lambdas' de Python para implementar las diferentes funciones.
En el caso de la curva logistica dada, recibe un instante de tiempo $ t $ asi como una constante $ k $ y $ M $ que son por lo tanto requeridas en la implementacion como parametro.

In [15]:
y = lambda k, M, t : (M / (1 + math.exp(-k * M * t)))

### 2) Implementar el algoritmo de Runge-Kutta de orden 4
Primeramente se explicara el metodo, luego su desarrollo para la problematica presentada

---
#### Ecuaciones 
$ y_{i+1}=y_i + \frac{h}{6} \cdot (k_1 + 2 \cdot k_2 + 2 \cdot k_3 + k_4) $

$ k_1 = f(x_i , y_i ) $

$ k_2 = f(x_i + \frac{h}{2}, y_i + \frac{k_1}{2}  ) $

$ k_3 = f(x_i + \frac{h}{2}, y_i + \frac{k_2}{2} ) $

$ k_4 = f(x_i + h, y_i + k_3 ) $

#### Development 1
Runge-Kutta me pide una funcion $ \frac{dy}{dx} $, y luego un valor inicial $ x_0 $ y $ y_0 $ asi como un valor $ h $ delta. 

Decimos entonces que $ \frac{dy}{dx} = f(x, y) $ y $ y(x_0) = y_0 $

In [22]:
y_deriv  = lambda k, M, t : k * y(k, M, t) * (M - y(k, M, t))  # calculates y from existing y(k, M, t)
y_deriv_2 = lambda k, M, t, yi: k * yi * (M - yi)  # uses calculated yi, which is given


y_next    = lambda k, M, t, yi, h: yi + h/6*(k1(k, M, t, yi) + 2*k2(k, M, t, yi, h) 
                                            + 2*k3(k, M, t, yi, h) + k4(k, M, t, yi, h))
k1        = lambda k, M, t, yi      : y_deriv_2(k, M, t, yi)
k2        = lambda k, M, t, yi, h   : y_deriv_2(k, M, t + (h / 2), yi + k1(k, M, t, yi) / 2)
k3        = lambda k, M, t, yi, h   : y_deriv_2(k, M, t + (h / 2), yi + k2(k, M, t, yi, h) / 2)
k4        = lambda k, M, t, yi, h   : y_deriv_2(k, M, t + h      , yi + k3(k, M, t, yi, h) / 2)

### 3) Calcular utilizando el algoritmo de Runge-Kutta el impacto de la publicidad en cada canal y horario en 10 instantes de tiempo. Generar archivos csv para cada instante con el impacto de la publicidad para canal y horario.
#### Development 2
En nuestro caso tenemos los valores de $k$ y $M$ en listas.
El valor inicial $y_0$ se calculara usando la ecuacion logistica en un instante $t=0$.
Iterando en los canales y horarios aplicamos el algoritmo para 10 instantes de tiempo de $t=i \ \forall \ i=0, ..., 9$.
Los valores calculados fueron guardados en archivos 'csv' por cada canal existente (se espera obtener 5 archivos).

In [39]:
# list in the way we want: by time instance
time_instances = [[[] for i in range(len(coef_k))] for i in range(10)]
print(time_instances)

def add_new_set_of_ys(ys, channel_index):
    for i in range(10):
#         print(time_instances)
        time_instances[i][channel_index].append(ys[i])
        
# runge kutta calculations
for i in range(len(coef_k)):
    k = float(coef_k[i][0])
    for M in tv_viewers_channels[i]:
        ys = []
        y0 = y(k, float(M), 0)
        ys.append(y0)
        for j in range(1, 10):
            yi = y_next(k, float(M), j, ys[j-1], 2)
            ys.append(yi)
        add_new_set_of_ys(ys, i)

for i in range(len(time_instances)):
    print(i)
    f = open("time_instance_"+str(i)+".csv", "w")
    for j in range(len(time_instances[i])):
        print(time_instances[i][j])
        f.write(",".join(str(x) for x in time_instances[i][j])+"\n")
    f.close()



[[[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []]]
0
[60000.0, 45000.0, 75000.0, 150000.0, 200000.0, 250000.0, 400000.0, 450000.0, 500000.0, 475000.0]
[70000.0, 30000.0, 60000.0, 200000.0, 250000.0, 250000.0, 350000.0, 500000.0, 450000.0, 425000.0]
[300000.0, 50000.0, 100000.0, 150000.0, 250000.0, 300000.0, 350000.0, 400000.0, 375000.0, 325000.0]
[31500.0, 41500.0, 56500.0, 122800.0, 321650.0, 366171.5, 416071.5, 350000.0, 425000.0, 350000.0]
[200000.0, 250000.0, 150000.0, 125000.0, 177000.0, 271500.0, 321600.0, 350000.0, 300000.0, 325000.0]
1
[74356.98584258916, 53086.364386727844, 97395.23851452564, 238356.88903914625, 354911.1136485376, 487888.10754194856, 968351.5134836736, 1149802.0684199028, 1339019.775390625, 1243546.857222281]
[74898.74993856892, 30899.95781534747, 63599.32518220217, 239916.91593905858, 312297.

### 4) Hacer para cada instante de tiempo un mapa de calor con el impacto de la publicidad de cada canal y horario.



In [10]:
import numpy as np
import matplotlib.pyplot as plt

# https://matplotlib.org/3.1.1/gallery/images_contours_and_fields/image_annotated_heatmap.html
for i in range(len(coef_k)):
    rows_horarios = ["h0", "h1", "h2", "h3", "h4", "h5", "h6", "h7", "h8", "h9"]
    cols_times = ["t0", "t1", "t2", "t3", "t4", "t5", "t6", "t7", "t8", "t9"]
    f = open("channel_"+str(i)+".csv", "r")
    matrix = []
    pre_matrix = f.readlines()
    for item in pre_matrix:
        new_item = item.split(",")
        row = []
        for subitem in new_item:
            row.append(float(subitem.rstrip()))
        matrix.append(row)
    matrix_2 = np.array(matrix)
    fig, ax = plt.subplots()
    im = ax.imshow(matrix_2)
    ax.set_xticks(range(len(cols_times)))
    ax.set_yticks(range(len(rows_horarios)))
    ax.set_xticklabels(cols_times)
    ax.set_yticklabels(rows_horarios)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", 
            rotation_mode="anchor")
    for k in range(len(cols_times)):
        for j in range(len(rows_horarios)):
            text = ax.text(j, k, matrix_2[k, j],
                          ha="center", va="center")
    
    ax.set_title("Channel " + str(i))
#     fig.tight_layout()
    plt.show()
            
    f.close()


FileNotFoundError: [Errno 2] No such file or directory: 'channel_0.csv'

### 5) Comparaciones entre los métodos de los puntos (1) con Runge-Kutta de orden 4 y (2).
TODO

### 6) Analizar los datos obtenidos y sugerir una posible distribución del dinero disponible para que el impacto de la publicidad sea máximo.
TODO

## Conclusion
TODO